## 🎮 Street Fighter con MediaPipe Pose  

El proyecto consiste en una **implementación interactiva del juego Street Fighter**, en la cual se emplea la tecnología de **MediaPipe Pose** para interpretar los **gestos corporales del usuario** y convertirlos en **comandos de control del personaje**.

### ⚙️ ¿Cómo funciona?  

Mediante el uso de **OpenCV** para la **captura de video**, **MediaPipe Pose** para la **detección y seguimiento de puntos clave del cuerpo**, y **PyDirectInput** para la **simulación programática de entradas del teclado**, el sistema permite que el jugador ejecute acciones dentro del juego utilizando exclusivamente los **movimientos de su cuerpo**.

De esta forma, se reemplaza el **esquema tradicional de control físico** por una interfaz de interacción basada en **detección de poses**, proporcionando una experiencia más **inmersiva, dinámica y natural** en el entorno de combate.

### 🕹️ Movimientos implementados  

- ✊ **Golpe Izq–Der** → Movimiento rápido del brazo hacia adelante.  
- 🦶 **Patada Izq–Der** → Elevación de la pierna a determinada altura.  
- 🚶 **Avanzar** → Desplazamiento corporal hacia adelante.  
- 🔙 **Retroceder** → Desplazamiento corporal hacia atrás.  
- 🕴️ **Saltar** → Detección de aumento repentino en la posición del cuerpo.  
- 🧎 **Agacharse** → Disminución notable de la altura del cuerpo (flexión de rodillas).  
- 🔥 **Hadouken** → Movimiento simultáneo de ambos brazos hacia adelante, simulando el clásico ataque de energía.  


Equipo
```
Aqué González José Manuel 
Obed Domínguez Mora
Ortiz Valle Leonardo Yeret
```

In [162]:
import mediapipe as mp
print(mp.__version__)


0.10.21


## ⭕ MediaPipe Pose - Skeleton KeyPoint 

<img style="padding-left:50px;" src="images/MediaPipePose.jpg">




In [163]:
import cv2
import mediapipe as mp
import pydirectinput
import time

In [164]:
def limites_saltar_agachar(y_nariz, y_hombro_izq, y_hombro_der):
    y_delta_hombro_nariz = abs(y_nariz - ((y_hombro_izq + y_hombro_der) / 2))
    
    limite_saltar = y_nariz - y_delta_hombro_nariz
    limite_agachar = 1.33 * (y_nariz + y_delta_hombro_nariz)
    
    return limite_saltar, limite_agachar

In [165]:
def limites_acerca_alejar(x_nariz, x_hombro_izq, x_hombro_der):
    x_hombro = (x_hombro_izq + x_hombro_der) / 2
    x_delta_hombro_nariz = abs(x_nariz - (x_hombro))
    
    limite_acercar = 0.9 * (x_hombro + x_delta_hombro_nariz)
    limite_alejar = 1.4 * (x_hombro - x_delta_hombro_nariz)
    
    return limite_acercar, limite_alejar

In [166]:
def limite_golpe_der(y_hombro_der, y_mano_der):
    longitud_brazo = abs(y_hombro_der - y_mano_der)
    x_limite_golpear = 0.55 * longitud_brazo
    return x_limite_golpear

In [167]:
def limite_golpe_izq(y_hombro_izq, y_mano_izq):
    longitud_brazo = abs(y_hombro_izq - y_mano_izq)
    x_limite_golpear = 0.55 * longitud_brazo
    return x_limite_golpear


In [ ]:
def iniciar_juego(cap):
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image_flipped = cv2.flip(frame, 1)

        cv2.putText(image_flipped, "Presiona 'i' para iniciar el control por pose", (10, 30),
                    cv2.FONT_HERSHEY_DUPLEX, 0.8, (50, 255, 50), 2)
        cv2.putText(image_flipped, "Presiona ']' para salir", (10, 60),
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 50, 50), 2)

        cv2.imshow("Iniciar Control por Pose", image_flipped)

        key = cv2.waitKey(1) & 0xFF   # <--- SOLO UNA VEZ

        if key == ord('i'):
            # temporizador visual
            for t in ['5','4','3', '2', '1']:
                temp = image_flipped.copy()
                cv2.putText(temp, t, (300, 300),
                            cv2.FONT_HERSHEY_DUPLEX, 2.0, (50,50,255), 4)
                cv2.imshow("Iniciar Control por Pose", temp)
                cv2.waitKey(1000)  # <--- 1000 ms = 1 segundo

            cv2.destroyWindow("Iniciar Control por Pose")
            juego(cap)
            break

        elif key == ord(']'):
            cv2.destroyWindow("Iniciar Control por Pose")
            break

        
def juego(cap):
    # --- Inicializar MediaPipe Pose (para detección de cuerpo) ---
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    mp_drawing_pose = mp.solutions.drawing_utils

    # --- Variables para acciones corporales ---
    accion = 'Neutra'  # Acción inicial
    movimiento = 'Neutro'  # Texto inicial
    proporcion_inicial_flag = False  # Flag para proporciones iniciales

    # Límites de movimiento
    limite_saltar = 0
    limite_agachar = 0
    limite_acercar = 0
    limite_alejar = 0

    # Estado de las teclas (True = presionada, False = liberada)
    estado_teclas = {
        'up': False, 'down': False, 'left': False, 'right': False,
        'alt': False, 'ctrlleft': False, 'z': False, 'x': False,
        'hadouken': False
    }
    # Bucle principal de control
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        ret, frame = cap.read()  # Leer frame de la cámara
        if not ret:
            break  # Salir si no se puede leer

        # Copiar frame original
        img = frame.copy()
        fraccion = 1.5
        # Redimensionar imagen (aquí no se cambia tamaño)
        image = cv2.resize(img, (0, 0), fx=fraccion, fy=fraccion, interpolation=cv2.INTER_NEAREST)
        # Convertir a RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Convertir a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Procesar pose
        results_pose = pose.process(image_rgb)

        # Si se detectan landmarks
        if results_pose.pose_landmarks:
            # Coordenadas verticales
            y_nariz = results_pose.pose_landmarks.landmark[0].y
            y_hombro_izq = results_pose.pose_landmarks.landmark[11].y
            y_hombro_der = results_pose.pose_landmarks.landmark[12].y
            # Coordenadas horizontales
            x_nariz = results_pose.pose_landmarks.landmark[0].x
            x_hombro_izq = results_pose.pose_landmarks.landmark[11].x
            x_hombro_der = results_pose.pose_landmarks.landmark[12].x
            # Coordenadas manos
            y_mano_der = results_pose.pose_landmarks.landmark[20].y
            x_mano_der = results_pose.pose_landmarks.landmark[20].x
            y_mano_izq = results_pose.pose_landmarks.landmark[19].y
            x_mano_izq = results_pose.pose_landmarks.landmark[19].x
            # Coordenadas pies
            x_pie_der = results_pose.pose_landmarks.landmark[28].x
            x_pie_izq = results_pose.pose_landmarks.landmark[27].x

        # --- Configurar proporciones iniciales una vez ---
        if proporcion_inicial_flag == False:
            limite_saltar, limite_agachar = limites_saltar_agachar(y_nariz, y_hombro_izq, y_hombro_der)
            limite_acercar, limite_alejar = limites_acerca_alejar(x_nariz, x_hombro_izq, x_hombro_der)
            x_limite_golpear_der = limite_golpe_der(y_hombro_der, y_mano_der)
            x_limite_golpear_izq = limite_golpe_izq(y_hombro_izq, y_mano_izq)
            proporcion_inicial_flag = True


        # ==================== DETECCIÓN DE POSE ====================
        
        if results_pose.pose_landmarks:
            # Dibujar landmarks y conexiones
            mp_drawing_pose.draw_landmarks(image_bgr, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Dibujar números de landmarks
            h, w, _ = image_bgr.shape
            for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                cv2.putText(image_bgr, str(idx), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1, cv2.LINE_AA)
                    
            # Calcular límite dinámico de golpe
            x_limite_para_golpe_der = x_hombro_der - x_limite_golpear_der
            x_limite_para_golpe_izq = x_hombro_izq - x_limite_golpear_izq
            
            #Dibujar lineas límete para golpe
            h, w = image_bgr.shape[:2]
            x_golpe_px = int(x_limite_para_golpe_der * w)
            cv2.line(image_bgr, (x_golpe_px, 0), (x_golpe_px, h), (255, 0, 0), 2)  # línea azul derecha
            x_golpe_px2 = int(x_limite_para_golpe_izq * w)
            cv2.line(image_bgr, (x_golpe_px2, 0), (x_golpe_px2, h), (0, 0, 255), 2)  # línea azul izquierda
            
            # Dibujar línea de salto (verde)
            y_js = limite_saltar
            if y_js < 0.0: y_js = 0.0
            if y_js > 1.0: y_js = 1.0
            y_js_px = int(y_js * h)
            cv2.line(image_bgr, (0, y_js_px), (w - 1, y_js_px), (0, 255, 0), 2)

            # Dibujar línea de agacharse (rojo)
            y_ag = limite_agachar
            if y_ag < 0.0: y_ag = 0.0
            if y_ag > 1.0: y_ag = 1.0
            y_ag_px = int(y_ag * h)
            cv2.line(image_bgr, (0, y_ag_px), (w - 1, y_ag_px), (0, 255, 255), 2)

            # Dibujar línea de acercar (cyan)
            x_ac = limite_acercar
            if x_ac < 0.0: x_ac = 0.0
            if x_ac > 1.0: x_ac = 1.0
            x_ac_px = int(x_ac * w)
            cv2.line(image_bgr, (x_ac_px, 0), (x_ac_px, h - 1), (255, 255, 0), 2)

            # Dibujar línea de alejar (magenta)
            x_al = limite_alejar
            if x_al < 0.0: x_al = 0.0
            if x_al > 1.0: x_al = 1.0
            x_al_px = int(x_al * w)
            cv2.line(image_bgr, (x_al_px, 0), (x_al_px, h - 1), (255, 0, 255), 2)
            
            # Promedio de hombros (para movimiento horizontal)
            # nariz_y = results_pose.pose_landmarks.landmark[0].y
            x11 = results_pose.pose_landmarks.landmark[11].x
            x12 = results_pose.pose_landmarks.landmark[12].x
            x_hombro = (x11 + x12) / 2
            
            # ==================== DETECCIÓN DE ACCIONES ====================

            # --- Movimiento vertical ---
            if y_nariz < limite_saltar:  # Saltar
                if not estado_teclas['up']:
                    pydirectinput.keyDown('up')
                    estado_teclas['up'] = True
                if estado_teclas['down']:
                    pydirectinput.keyUp('down')
                    estado_teclas['down'] = False
                movimiento = "Saltar"
            elif y_nariz > limite_agachar:  # Agacharse
                if not estado_teclas['down']:
                    pydirectinput.keyDown('down')
                    estado_teclas['down'] = True
                if estado_teclas['up']:
                    pydirectinput.keyUp('up')
                    estado_teclas['up'] = False
                movimiento = "Agachar"
            else:  # Neutro
                if estado_teclas['up']:
                    pydirectinput.keyUp('up')
                    estado_teclas['up'] = False
                if estado_teclas['down']:
                    pydirectinput.keyUp('down')
                    estado_teclas['down'] = False

            # --- Movimiento horizontal ---
            if x_hombro < limite_acercar:  # Acercar
                if not estado_teclas['right']:
                    pydirectinput.keyDown('right')
                    estado_teclas['right'] = True
                if estado_teclas['left']:
                    pydirectinput.keyUp('left')
                    estado_teclas['left'] = False
                movimiento = "Acercar"
            elif x_hombro > limite_alejar:  # Alejar
                if not estado_teclas['left']:
                    pydirectinput.keyDown('left')
                    estado_teclas['left'] = True
                if estado_teclas['right']:
                    pydirectinput.keyUp('right')
                    estado_teclas['right'] = False
                movimiento = "Alejar"
            else:  # Neutro
                if estado_teclas['left']:
                    pydirectinput.keyUp('left')
                    estado_teclas['left'] = False
                if estado_teclas['right']:
                    pydirectinput.keyUp('right')
                    estado_teclas['right'] = False
                    
            if x_hombro < limite_alejar and x_hombro > limite_acercar and y_nariz > limite_saltar and y_nariz < limite_agachar:  # Neutro
                movimiento = "Neutro"     
            
            
                
            # --- Golpe especial: Hadouken ---
            if x_mano_der < x_limite_para_golpe_der and x_mano_izq < x_limite_para_golpe_izq:
                if not estado_teclas['hadouken']:
                    # Secuencia ↓ → ↘ + puño
                    pydirectinput.keyDown('down')
                    pydirectinput.keyDown('right')
                    pydirectinput.keyUp('down')
                    pydirectinput.keyDown('ctrlleft')
                    pydirectinput.keyUp('right')
                    pydirectinput.keyUp('ctrlleft')
                    estado_teclas['hadouken'] = True
                accion = "Hadouken"
            else:
                estado_teclas['hadouken'] = False

            # --- Golpes individuales de manos ---
            if x_mano_der < x_limite_para_golpe_der:
                if not estado_teclas['alt']:
                    pydirectinput.keyDown('alt')
                    estado_teclas['alt'] = True
                accion = "Golpear - Alt"
            else:
                if estado_teclas['alt']:
                    pydirectinput.keyUp('alt')
                    estado_teclas['alt'] = False
                
            if x_mano_izq < x_limite_para_golpe_izq:
                if not estado_teclas['ctrlleft']:
                    pydirectinput.keyDown('ctrlleft')
                    estado_teclas['ctrlleft'] = True
                accion = "Golpear - Ctrl-left"
            else:
                if estado_teclas['ctrlleft']:
                    pydirectinput.keyUp('ctrlleft')
                    estado_teclas['ctrlleft'] = False
            
            # --- Golpes con pies ---
            if x_pie_izq < x_limite_para_golpe_izq:
                if not estado_teclas['z']:
                    pydirectinput.keyDown('z')
                    estado_teclas['z'] = True
                accion = "Patear - Z"
            else:
                if estado_teclas['z']:
                    pydirectinput.keyUp('z')
                    estado_teclas['z'] = False
                    
            if x_pie_der < x_limite_para_golpe_der:
                if not estado_teclas['x']:
                    pydirectinput.keyDown('x')
                    estado_teclas['x'] = True
                accion = "Patear - X"
            else:
                if estado_teclas['x']:
                    pydirectinput.keyUp('x')
                    estado_teclas['x'] = False
                    
        # ==================== MOSTRAR EN PANTALLA ====================
        image_flipped = cv2.flip(image_bgr, 1)  # Voltear imagen horizontal
        cv2.putText(image_flipped, f"Movimiento: {movimiento}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(image_flipped, f"Golpe: {accion}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Pose + Hand Tracking", image_flipped)

        if key == ord(']'):  # Salir si se presiona ']'
            cv2.destroyWindow("Pose + Hand Tracking")
            iniciar_juego(cap)
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [169]:
cap = cv2.VideoCapture(1) # Captura de cámara externa
iniciar_juego(cap)